Imports

In [1]:
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve

import http.client as httplib 

from astropy.table import Table,join
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)
import SciServer
from SciServer import CasJobs, SkyQuery, SciDrive, SkyServer
import requests
import pandas as pd
import astropy as ap
import astropy.io.fits as astrofits
import astropy.io.votable as astrovot
import astropy.wcs as astrowcs
import astropy.units as u
import astropy.coordinates as astrocoords
import astropy.visualization as astrovis
import astropy.visualization.mpl_normalize as astromplnorm
import astropy.nddata as astronddata
import astropy.nddata.utils as astrondutils
import io
import pdb 
from IPython.core.display import display
from astropy.table import Table,vstack,Column
from astroquery.mast import Catalogs
from astropy.utils.console import ProgressBar
from collections import deque
from pydoc import locate

import matplotlib.pyplot as mplplot

import gzip
import tarfile
import math
import shutil
from astropy.nddata.utils import Cutout2D

Definitions

In [2]:
#Query the MAST API
def mastQuery(request):
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    executing = True
    while executing:
        # Making the query
        conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

        # Getting the response

        resp = conn.getresponse()
        head = resp.getheaders()
        content = resp.read().decode('utf-8')
        executing = 'EXECUTING' in content
        print('Still executing')
    # Close the https connection
    conn.close()

    return head,content


#Crossmatch between a catalog and the HSC catalog
def HscCrossmatch(data):
    crossmatchInput = {"fields":[{"name":"ra","type":"float"},
                                 {"name":"dec","type":"float"}],
                       "data":data}
    request = {"service":"Mast.Hsc.Crossmatch.MagAper2v3",
                   "data":crossmatchInput,
                   'params':{
                       "raColumn":"ra",
                       "decColumn":"dec",
                       'radius':0.0001,
                       'cache-breaker':10},
                   "pagesize":1000,
                   "page":1,
                   "format":"json",
                   "removecache":True}
        
    headers,outString = mastQuery(request)
    outData = json.loads(outString)
    
    return outData


#Get more information about the crossmatched galaxies
def getHSCMatches(matchId):
    request = {'service':'Mast.HscMatches.Db.v3',
               'params':{'input':matchId,
                         'cache-breaker':10},
               'format':'json',
               'page':1,
               'pagesize':4}   

    headers,outString = mastQuery(request)

    outData = json.loads(outString)

    return outData


#Hugh's code used to generate the correct url for downloading files from the HSC catalog
def genBundleRequest(imageNames, outfileNamePrefix='downloadBundle', extension='tar.gz'):
    baseUrl = "http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset="
    urlList = ",".join(['{}{}'.format(baseUrl, imageName) for imageName in imageNames])
    pathList = ['{}.fits'.format(imageName) for imageName in imageNames]

    request = {"service":"Mast.Bundle.Request",
               "params":{"urlList":urlList,
                         "filename":outfileNamePrefix,
                         "pathList":pathList,
                         "extension":extension,
                         'cache-breaker':10}
              }
    return request,outfileNamePrefix,extension


#Takes the output from the Bundle Request and downloads the files at that url
def downloadRequest(url):
    server='mast.stsci.edu'
    
    conn = httplib.HTTPSConnection(server)
    
    while True:
        conn.request("GET", url)
        resp = conn.getresponse()
        try:
            fileName = resp.getheader('Content-Disposition')[21:]
            fileContent = resp.read()
            with open(fileName,'wb') as FLE:
                FLE.write(fileContent)
            conn.close()

            return fileName
        except TypeError as e:
            print('Caught TypeError {}: {}, {}'.format(e, resp.getheader('Content-Disposition'), resp.read()))


    

#The following are with the help of Michael

#Reassign the datatypes from the json file to the correct datatype
def stringtype(typestr):
    typemap = {'string':str,'float':np.float64,'int':int,'boolean':bool,'date':str}
    return typemap[typestr]
    

#Create strings from values
def typecast(val,typestr):
    if val is None:
        return None
    elif typestr == 'string':
        return str(val)
    else:
        return locate(typestr)(val)
    

#Create astropy tables from the json files masking all values that create errors
def mastJson2Table(line):
    jsonObj = line
    if not jsonObj['data']:
        return None
    coldict = {field['name']:stringtype(field['type']) for field in jsonObj['fields']}
    keys,dtypes = zip(*coldict.items())
    
    rows = deque()
    
    for d in jsonObj['data']:
        row = [d[key] if d[key] is not None else '99' for key in keys]
        rows.append(row)
     
    table = Table(rows=list(rows),names=keys,dtype=dtypes)
    return table


#Stack tables together in one table
def read_matches(lines):
    tables = deque()
    for line in ProgressBar(lines):
        table = mastJson2Table(line)
        if table:
            tables.append(table)
    table = vstack(list(tables))
    return table



Catalogs

In [3]:
#SDSS Catalog
query = "select zoo2MainSpecz.dr8objid, Galaxy.ra, Galaxy.dec, Galaxy.petroR90_r, Galaxy.petroR90Err_r  FROM zoo2MainSpecz JOIN Galaxy ON zoo2MainSpecz.dr8objid = Galaxy.ObjID"
result = CasJobs.executeQuery(query, 'DR10', format='pandas')
ra_SDSS = result.ra.values
dec_SDSS = result.dec.values
obj = result.dr8objid.values
petroR90 = result.petroR90_r.values
petroR90Err_r = result.petroR90Err_r.values
display(result)

,dr8objid,ra,dec,petroR90_r,petroR90Err_r
0,1237648704044728453,193.670672,-0.392065,6.863250,0.176251
1,1237648704044794076,193.876571,-0.322842,6.525316,0.199878
2,1237648704044925049,194.112992,-0.388663,7.053779,0.165174
3,1237648704044925082,194.143390,-0.221734,6.747866,0.175773
4,1237648704044925105,194.159925,-0.224336,6.837379,0.114140
5,1237648704044990592,194.308629,-0.329539,9.313372,0.175340
6,1237648704045318156,195.015800,-0.370944,7.383853,0.303520
7,1237648704045318338,195.136286,-0.365312,9.167497,0.309375
8,1237648704045383818,195.230877,-0.235435,6.450408,0.130637
9,1237648704045908141,196.448765,-0.385815,6.229260,0.107333


In [29]:
print(obj[187799])

1237668494719975453


In [4]:
#HSC Crossmatche Catalog
os.chdir('/home/idies/workspace')
bins = 500
size = int(len(ra_SDSS)/500)
lines = []
for i in range(int(size)):
    ra_dec_list = [{"ra":ras,"dec":decs} for ras,decs in zip(ra_SDSS,dec_SDSS)]
    data = ra_dec_list[i*bins:(i*bins)+bins]
    info = HscCrossmatch(data)
    #display(info)
    lines.append(info)
    if i%50 == 0:
        print('running')
stacked_table = read_matches(lines)
print(stacked_table['MatchRA'][0])


Still executing
running
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
running
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still ex

In [5]:
place = 2
print(stacked_table['ra'][place])
print(stacked_table['MatchRA'][place])

132.437019361891
132.43703985792766


In [6]:
coords_HSC = astrocoords.SkyCoord(ra=stacked_table['MatchRA'],dec=stacked_table['MatchDec'],unit=(u.deg,u.deg))
coords_SDSS = astrocoords.SkyCoord(ra_SDSS,dec_SDSS,unit=(u.deg,u.deg))

idx, d2d, d3d = coords_HSC.match_to_catalog_sky(coords_SDSS)
#Add things to table
objectid = Column(name='dr8objid', data=[obj[x] for x in idx])
stacked_table.add_column(objectid)

In [ ]:
stacked_table.write('test.fits')
stacked_table.write('test.ecsv')

Download and Delete

In [7]:
filt = 'A_F475W'
matchIDs = [i['MatchID'] for i in stacked_table if isinstance(i[filt], np.float64) and i[filt] != 99.0 and not math.isnan(i[filt])]
images = [mastJson2Table(getHSCMatches(str(i)))['ImageName'] for i in matchIDs]

image_dict = {}
for i,name in enumerate(images):
    image_list = [j for j in name if 'f475w' in j]
    for k in image_list:
        image_dict[k] = [matchIDs[ind] for ind,out in enumerate(images) for In in out if In == k]

Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still ex

In [8]:
os.chdir('/home/idies/workspace/persistent/Images')
name_list = []
for number,key in enumerate(image_dict):
    name_list.append(key)
query,filename,extension = genBundleRequest(name_list[:4])
pp.pprint(query)
print('1')
headers,bundleString = mastQuery(query)
bundleInfo = json.loads(bundleString)
pp.pprint(bundleInfo)
print('2')
downloadfile = downloadRequest(bundleInfo['url'])
tar = tarfile.open("downloadBundle.tar.gz")
os.chdir('/home/idies/workspace/persistent/Images/Images')
tar.extractall()
tar.close()
os.remove('System.String[]')
shutil.rmtree('downloadBundle')

{   'params': {   'cache-breaker': 10,
                  'extension': 'tar.gz',
                  'filename': 'downloadBundle',
                  'pathList': [   'hst_10861_75_acs_wfc_f475w.fits',
                                  'hst_9401_65_acs_wfc_f475w.fits',
                                  'hst_14361_02_acs_wfc_f475w.fits',
                                  'hst_10861_09_acs_wfc_f475w.fits'],
                  'urlList': 'http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_10861_75_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_65_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_14361_02_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_10861_09_acs_wfc_f475w'},
    'service': 'Mast.Bundle.Request'}
1
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
{   'bytesStreamed': 1681293326,
    'fileStatusList': {   'http://hla.stsci.edu

Make Cutouts

In [9]:
os.chdir('/home/idies/workspace/persistent/Images/Images')
for filename in os.listdir('/home/idies/workspace/persistent/Images/Images'):
    if filename == '.ipynb_checkpoints':
        continue
    print(filename)
    hdu = astrofits.open(filename)
    w = astrowcs.WCS(hdu[1].header)
    print(w)
    name = filename.replace('.fits', '')
    for number,z in enumerate(image_dict[name]):
        for i,j in enumerate(stacked_table['MatchID']):
            if j == z:
                ra_cut = stacked_table['MatchRA'][i]
                dec_cut = stacked_table['MatchDec'][i]
                coord = astrocoords.SkyCoord(ra_cut,dec_cut,unit=(u.deg,u.deg),frame='icrs')
                print('{:.10f}, {:.10f}'.format(ra_cut,dec_cut))
                cut = Cutout2D(hdu[1].data,position=coord,size=4*u.arcsec,wcs=w)
                cut_fits = astrofits.PrimaryHDU(data=cut.data, header=cut.wcs.to_header())
                cut_fits.writeto('/home/idies/workspace/persistent/Images/Cuts/'+name+'_'+str(number)+'_cut.fits')

hst_10861_09_acs_wfc_f475w.fits
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 195.1156285764077  28.02847902262505  
CRPIX : 2542.499999999837  2513.999999999839  
CD1_1 CD1_2  : -1.3888888888888e-05  -2.4356242949037e-35  
CD2_1 CD2_2  : 0.0  1.38888888888896e-05  
NAXIS : 5084  5027
195.0922513143, 28.0469975955
hst_14361_02_acs_wfc_f475w.fits
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 195.0495716513231  27.87566361552696  
CRPIX : 2667.499999999829  2640.999999999831  
CD1_1 CD1_2  : -1.3888888888888e-05  4.87124858980755e-35  
CD2_1 CD2_2  : 0.0  1.38888888888896e-05  
NAXIS : 5334  5281
195.0380908677, 27.8664495136
195.0426406018, 27.8639431675
195.0567902626, 27.8671829298
hst_9401_65_acs_wfc_f475w.fits
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 188.4697693371468  13.32126810735623  
CRPIX : 2599.499999999834  2590.999999999834  
CD1_1 CD1_2  : -1.3888888888888e-05  0.0  
CD2_1 CD2